In [1]:
%autosave 10

Autosaving every 10 seconds


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl

In [3]:
# Import the Dataset
df = pd.read_csv('E:\Technolabs\cleaned_final.csv')

In [4]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month', 'EDUCATION_CAT', 'graduate school',
       'high school', 'others', 'university'],
      dtype='object')

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df[['PAY_1','LIMIT_BAL']].values,df['default payment next month'].values,test_size=0.2,random_state=24)

In [10]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(21331, 2)
(5333, 2)
(21331,)
(5333,)


In [11]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear')

In [12]:
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
#Predict on test data
y_pred = lr.predict(X_test)

In [15]:
# Let's get the predicted probabilities
pred_proba = lr.predict_proba(X_test)
pred_proba

array([[0.74826924, 0.25173076],
       [0.584297  , 0.415703  ],
       [0.79604453, 0.20395547],
       ...,
       [0.584297  , 0.415703  ],
       [0.82721498, 0.17278502],
       [0.66393435, 0.33606565]])

In [16]:
print(lr.coef_,lr.intercept_)

[[ 8.27451187e-11 -6.80876727e-06]] [-6.57647457e-11]


In [17]:
np.ones((X_test.shape[0],1)).shape

(5333, 1)

In [19]:
ones_and_features = np.hstack([np.ones((X_test.shape[0],1)), X_test])
ones_and_features.shape

(5333, 3)

In [23]:
intercept_and_coefs = np.concatenate([lr.intercept_.reshape(1,1), lr.coef_], axis=1)
intercept_and_coefs

array([[-6.57647457e-11,  8.27451187e-11, -6.80876727e-06]])

In [25]:
X_lin_comb = np.dot(intercept_and_coefs, np.transpose(ones_and_features))
X_lin_comb

array([[-1.08940276, -0.34043836, -1.36175345, ..., -0.34043836,
        -1.56601647, -0.68087673]])

### Let's define a sigmoid function

In [29]:
def sigmoid(X):
    Y = 1 / (1 + np.exp(-X))
    return Y

In [30]:
y_pred_proba_manual = sigmoid(X_lin_comb)

In [34]:
# Let's set sigmoid value as 0.5
y_pred_manual = y_pred_proba_manual >= 0.5
y_pred_manual

array([[False, False, False, ..., False, False, False]])

In [32]:
y_pred.shape

(5333,)

In [33]:
y_pred_manual.shape

(1, 5333)

In [35]:
#Let's compare it with pred class
np.array_equal(y_pred.reshape(1,-1), y_pred_manual)

True

#### Let's calculate the ROC_AUC 

In [37]:
from sklearn.metrics import roc_auc_score

In [41]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
roc_auc_score(y_test, pred_proba[:,1])

0.627207450280691

In [42]:
# Use manually calculated predicted probabilities to calculate ROC AUC
roc_auc_score(y_test, y_pred_proba_manual.reshape(y_pred_proba_manual.shape[1],))

0.627207450280691